In [1]:
import numpy as np
import math
TRAIN_FILE_X = 'data/q1/train_x.txt'
TRAIN_FILE_Y = 'data/q1/train_y.txt'
TEST_FILE_X = 'data/q1/test_x.txt'
TEST_FILE_Y = 'data/q1/test_y.txt'
VAL_FILE_X = 'data/q1/valid_x.txt'
VAL_FILE_Y = 'data/q1/valid_y.txt'

In [2]:
def readsparse(filename):
    data = []
    i = 0
    with open(filename) as f:
        for line in f:
            if(len(data)==0):
                head = line.split()
                data = np.zeros((int(head[0]),int(head[1]))).astype(float)
                i = 0
            else:
                for el in line.split():
                    val = float(el.split(':')[1])
                    ind = int(el.split(':')[0])
                    data[i][ind]=val
                i+=1
    return data

#### Load data

In [3]:
X = readsparse(TRAIN_FILE_X)
Y = np.genfromtxt(TRAIN_FILE_Y)

In [4]:
vX = readsparse(VAL_FILE_X)
vY = np.genfromtxt(TRAIN_FILE_Y)

In [5]:
tX = readsparse(TEST_FILE_X)
tY = np.genfromtxt(TEST_FILE_Y)

### Tree DS

In [9]:
def getEntropyforAttribute(X,Y,i):
    x = X[:,i]
    median = np.median(x)
    right = np.where(x > median)[0]
    left  = np.where(x <= median)[0]
    
    # If the attribute has no records / all records are same
    if(len(left)==0 or len(right)==0):
        return 10.0
    
    # Number of instances which are 1
    left1 = len(np.where(Y[left] == 1)[0])
    right1 = len(np.where(Y[right] == 1)[0])
    
    # Percentage of 1s using this classification
    leftP = float(left1)/float(len(left))
    rightP = float(right1)/float(len(right))
    
    # Calculating entropy
    S = 0
    if(leftP > 0):
        S -= leftP * math.log(leftP)
    if(rightP > 0):
        S -= rightP * math.log(rightP)
    return S/math.log(2)

In [10]:
numAttr = X.shape[1]

In [11]:
def maxEntropyAttribute(X,Y,removedattributes):
    attr = -1;
    minS = 1000;
    for i in range(0,numAttr):
        if i not in removedattributes:
            S = getEntropyforAttribute(X,Y,i);
            if (S < minS):
                minS = S;
                attr = i;
    return attr

### Tree Data structure

In [85]:
class Tree:
    def __init__(self):
        self.left = None
        self.right = None
        self.val = 0
        self.attr = -1
        self.med = -1
        self.ht = -1
    def setVal(self,Y):
        if(np.mean(Y) > 0.5): self.val = 1
        else: self.val = 0
        return
    def build(self,X,Y,ht,removedattributes):
        if(len(X) < 10 or ht > htlim):
            ## This would be a leaf
            self.attr=-2
            self.setVal(Y)
#             print("this was a leaf")
            return
        Attribute = maxEntropyAttribute(X,Y,removedattributes)
        if(Attribute == -1):
            return
        self.attr = Attribute
        self.med = np.median(X[:,self.attr])
        self.ht = ht
        removedattributes.append(self.attr)
        left = np.where(X[:,self.attr] <= self.med)[0]
        self.left = Tree()
        self.left.build(X[left],Y[left],ht+1,removedattributes)
        right = np.where(X[:,self.attr] > self.med)[0]
        self.right = Tree()
        self.right.build(X[right],Y[right],ht+1,removedattributes)
    def printNode(self):
        print(self.attr,self.med,self.val)
    def printTree(self):
        self.printNode()
        if(self.left != None):
            print("<-")
            self.left.printTree()
        if(self.right != None):
            print("->")
            self.right.printTree()
    def predict(self,X):
        if(self.attr==-2):
            return self.val
        elif(self.attr==-1):
            return -1
        else:
            if(X[self.attr]>self.med):
                return self.right.predict(X)
            else:
                return self.left.predict(X)

In [87]:
htlim = 25

In [90]:
root = Tree()
# root.indices = np.arrange(X.shape[0])
root.build(X,Y,0,[])

In [91]:
R = np.ones_like(tY)
for i in range(0,tX.shape[0]):
    R[i] = root.predict(tX[i])
Correct = len(np.where(R == tY)[0])

In [92]:
print(float(Correct)/float(R.shape[0]))

0.606230587363
